In [3]:
def llm(msg, model):
    ans = client.chat.completions.create(
    model=model,
    messages=msg,
    max_tokens=1024,
    temperature=0.00001,
    seed=43000,
    stream=False)
    return ans.choices[0].message.content


In [14]:
system = """Your task is to complete FIM code, filling in only the middle section of the code. It will be provided to you in the following format: <fim_prefix>starting code</fim_prefix><fim_suffix>ending code</fim_suffix><fim_middle>, where you only need to complete the code in the middle section. 
For example:
Given the input: <fim_prefix># test.py
def test():
    print('hell</fim_prefix><fim_suffix>')</fim_suffix>
The output should be: <fim_middle>o world</fim_middle>"""
print(system)

Your task is to complete FIM code, filling in only the middle section of the code. It will be provided to you in the following format: <fim_prefix>starting code</fim_prefix><fim_suffix>ending code</fim_suffix><fim_middle>, where you only need to complete the code in the middle section. 
For example:
Given the input: <fim_prefix># test.py
def test():
    print('hell</fim_prefix><fim_suffix>')</fim_suffix>
The output should be: <fim_middle>o world</fim_middle>


In [15]:
prefix = """// AssetBundleGeneratorV2.cs
using System;
using System.Collections;
using System.Collections.Generic;
using UnityEditor;
using UnityEngine;
using System.IO;
using System.Linq;
using Newtonsoft.Json;
using UnityEditor.Build.Content;
using UnityEditor.Build.Pipeline;
using UnityEditor.Build.Pipeline.Interfaces;
using UnityEngine.Build.Pipeline;
using UnityEditor.Build.Pipeline.Tasks;
// using UnityEditor.AddressableAssets.Build.BuildPipelineTasks;
using UnityEngine.Networking;
using BuildCompression = UnityEngine.BuildCompression;

namespace NexgenDragon
{
	public partial class AssetBundleGeneratorV2 : AssetBundleGenerator
	{
		public static string OldBundlePath = "ArtData/OldBundles";
		public static int gHashLen = "d2e9405be3d60bbd575c97e8f33d".Length;

		public static bool miniMode = false;
		//	public static readonly string ASSETS_PATH_CONFIG_FILE = "assets_path_{0}.json";
		
		class CustomBuildParameters : BundleBuildParameters
		{
			public Dictionary<string, BuildCompression> PerBundleCompression { get; set; }

			public CustomBuildParameters(BuildTarget target, BuildTargetGroup group, string outputFolder) : base(target, group, outputFolder)
			{
				PerBundleCompression
"""

suffix = """
// Override the GetCompressionForIdentifier method with new logic
			public override BuildCompression GetCompressionForIdentifier(string identifier)
			{
				BuildCompression value;
				if (PerBundleCompression.TryGetValue(identifier, out value))
					return value;
				return BundleCompression;
			}
		}

		private static void CompressBundles(string bundleType)
		{
			var bundlePath = string.Format(AssetBundleOutputPath, bundleType);
			
			var files = Directory.GetFiles(bundlePath);
			foreach (var bundle in files)
			{
				if (AssetConfig.BundleNeedDecompress(bundle) && !bundle.EndsWith(".meta"))
				{
					var tmpName = bundle + "_temp";
					File.Move(bundle,tmpName);
					var ret = LzmaLib.encodeLzmaFile(tmpName, bundle, 9, 20);
					File.Delete(tmpName);
					if (ret != 0)
					{
						NLogger.Error("bundle {0} compress error happened",bundle);
						return;
					}
				}
			}
		}

		public static void DeleteDirectory(string target_dir) {
            string[] files = Directory.GetFiles(target_dir);
            string[] dirs = Directory.GetDirectories(target_dir);
            foreach (string file in files) {
                NLogger.Log("AssetBundleGeneratorV2.DeleteDirectory filename = {0}", file);
                File.SetAttributes(@file, FileAttributes.Normal);
                File.Delete(file);
            }
            foreach (string dir in dirs) {
                DeleteDirectory(dir);
            }
            Directory.Delete(target_dir, false);
        }
		
		[MenuItem("NexgenDragon/AssetBundleToolV2/GenerateAssetBundleForAndroid",false,800)]
		public static void GenerateAssetBundleForAndroidTool()
		{
			GenerateAssetBundleForAndroid();
		}

        [MenuItem("NexgenDragon/AssetBundleToolV2/TestGenerateAssetBundleForAndroid",false,801)]
        public static void GenerateAssetBundleForAndroidTool2()
        {
	        MoBuildReport.Init("1.0.0");
			var target = BuildTarget.Android;
            AssetBundleOutputPath = ANDROID_BUNDLE_PATH;
            var buildGroup = BuildPipeline.GetBuildTargetGroup(target);
            // Construct the new parameters class
            var buildParams = new CustomBuildParameters(target, buildGroup, GetAssetBundleFolderPath(BundleType.G));
            AssetBundleGeneratorV2.miniMode = true;
            var bundleBuilds = ProcessFolderBundleTag2.CollectBundle(target, buildParams);
            
            //use File Name addressableNames
            _allAddressNames.Clear();
            for (var i = 0; i < bundleBuilds.Length; i++)
            {
	            bundleBuilds[i].addressableNames = GetAddressableNamesList(bundleBuilds[i].assetNames).ToArray();
            }
            var buildContent = new BundleBuildContent(bundleBuilds);
            // Populate the bundle specific compression data
            //static和缓存的bundle用lzma压缩
            foreach (var build in bundleBuilds)
            {
	            if (AssetConfig.BundleNeedDecompress(build.assetBundleName))
	            {
		            buildParams.PerBundleCompression.Add(build.assetBundleName,BuildCompression.Uncompressed);
	            }
            }
        }
        public static void DeletePath(string fullPath)
        {
            string locationPathName = Path.GetFullPath(".") + "/../../build/";
            if (Directory.Exists(fullPath))
            {
	            Directory.Delete(fullPath, true);
	            return;
#if UNITY_EDITOR_WIN
                using var process = new System.Diagnostics.Process
                {
                    StartInfo = new System.Diagnostics.ProcessStartInfo
                    {
                        RedirectStandardOutput = true,
                        UseShellExecute = false,
                        CreateNoWindow = true,
                        WindowStyle = System.Diagnostics.ProcessWindowStyle.Hidden,
                        FileName = Path.Combine(locationPathName, "../buildtool/delete.bat"),
                        Arguments = fullPath
                    }

                };
                process.ErrorDataReceived += Process_ErrorDataReceived;
                process.Start();
                process.WaitForExit();             
#else
	            Directory.Delete(fullPath, true);
#endif
            }
        }

        private static void Process_ErrorDataReceived(object sender, System.Diagnostics.DataReceivedEventArgs e)
        {
			Debug.LogError("delete.bat error happened");
			Debug.LogError(e.ToString());
        }

        public new static void GenerateAssetBundleForAndroid(string resVersion = "0.0.0.0")
		{
			
#if UNITY_2020_2_OR_NEWER && !USE_SPRITE_ATLAS
			EditorSettings.spritePackerMode = SpritePackerMode.AlwaysOn;
			UnityEditor.Sprites.Packer.RebuildAtlasCacheIfNeeded(BuildTarget.Android, false);
#endif
			string bundlePath = Application.dataPath + "/StreamingAssets/GameAssets/AssetBundle2";
			DeletePath(bundlePath);
			
			AssetBundleOutputPath = ANDROID_BUNDLE_PATH;
			
			CompatibilityAssetBundleManifest m = BuildBundle(BuildTarget.Android);
			
			AppendExtension();

			MoveOtaToR();

			GenerateAssetBundleImplement(BundleType.R, m);

			MoveWwiseBankToG("Android");
			
			MoveCustomBundleToG("Android");
			
			GenerateAssetBundleImplement(BundleType.G, m);

			//强制使用旧版本bundle环节
			ProcessCertainPrefabs(true);

            AssetDatabase.Refresh();
            OnBundleBuilt();
        }

"""

In [17]:
msg = [
    {"role": "system", "content": system},
    {'role':'user', 'content': f"""Given the input: <fim_prefix>{prefix}</fim_prefix><fim_suffix>{suffix}</fim_suffix>
Skip the preamble, go straight into the answer.
The output should be:
"""}
]
print(llm(msg, 'claude-3-sonnet'))

<fim_middle>= new Dictionary<string, BuildCompression>();
			}
		}

		private static void GenerateAssetBundleImplement(string bundleType, CompatibilityAssetBundleManifest m)
		{
			var bundlePath = string.Format(AssetBundleOutputPath, bundleType);
			if (Directory.Exists(bundlePath))
			{
				Directory.Delete(bundlePath, true);
			}
			Directory.CreateDirectory(bundlePath);
			
			var buildParams = new CustomBuildParameters(BuildTarget.Android, BuildPipeline.GetBuildTargetGroup(BuildTarget.Android), bundlePath);
			buildParams.UseClassicLiveUploadIfPossible = false;
			buildParams.BundleCompression = BuildCompression.LZ4;
			buildParams.BundleCompressionLevel = 9;
			buildParams.BundleCompressionMaxDecompressionMemory = 20 * 1024 * 1024;
			buildParams.BundleCompressionMaxOutputLength = 20 * 1024 * 1024;
			buildParams.BundleCompressionMaxOutputRatio = 0.9f;
			buildParams.BundleCompressionMaxOutputFileSize = 20 * 1024 * 1024;
			buildParams.BundleCompressionMaxInputFileSize = 20 * 102

In [10]:
print(llm(msg, 'claude-3-haiku'))

Here is the completed code for the middle section:

<fim_middle>
= { };
			}
		}

		private static void CompressBundles(string bundleType)
		{
			var bundlePath = string.Format(AssetBundleOutputPath, bundleType);
			
			var files = Directory.GetFiles(bundlePath);
			foreach (var bundle in files)
			{
				if (AssetConfig.BundleNeedDecompress(bundle) && !bundle.EndsWith(".meta"))
				{
					var tmpName = bundle + "_temp";
					File.Move(bundle,tmpName);
					var ret = LzmaLib.encodeLzmaFile(tmpName, bundle, 9, 20);
					File.Delete(tmpName);
					if (ret != 0)
					{
						NLogger.Error("bundle {0} compress error happened",bundle);
						return;
					}
				}
			}
		}

		public static void DeleteDirectory(string target_dir) {
            string[] files = Directory.GetFiles(target_dir);
            string[] dirs = Directory.GetDirectories(target_dir);
            foreach (string file in files) {
                NLogger.Log("AssetBundleGeneratorV2.DeleteDirectory filename = {0}", file);
   

In [11]:
print("""
= new Dictionary<string, BuildCompression>();
			}
""")


= new Dictionary<string, BuildCompression>();
			}



In [18]:

aa = c.completions.create(model='FIM-Model',
                          prompt=prefix,
                          suffix=suffix)
print(aa.choices[0].text)

= new Dictionary<string, BuildCompression>();
			}
